In [70]:
import pandas as pd
import pickle
import json
import re
import ast

In [72]:
mandatory_dict_keys = ["Beispiel","Beispiel_für","Erzählposition","Wiedergabeform","in_gedankengang","Modus_Zeit_Beispiel","Modus_Zeit_Text"]

def extract_dict(string: str):
    """
    Extracts dictionary from the input string.

    Args:
        string (str): The input string.

    Returns:
        Union[str, None]: Extracted dictionary as a string, or None if no dictionary found.
    """
    start: int = string.find('{')
    end: int = string.find('}')
    if start != -1 and end != -1:
        return string[start:end+1]
    else: 
        return None
    
def correct_annotations(text):
    lines = text.split('\n')
    corrected_lines = []
    for line in lines:
        if '"Wiedergabeform"' in line and 'Erzählstimme' in line and ('indirekt' in line or 'direkt' in line):
            line = line.replace('Erzählstimme', 'Figurenrede')
            
        if '"Wiedergabeform"' in line and ('direkt' in line or 'indirekt' in line) and 'Figurenrede' not in line:
            line = line.replace('"Wiedergabeform": "', '"Wiedergabeform": "Figurenrede, ')

        corrected_lines.append(line)
    
    # Joining the corrected lines into a single formatted string
    formatted_result = '\n'.join(corrected_lines)
    return formatted_result

def create_response(response,check_mandatory=False):   
        
    response_dict = extract_dict(response)

    try: 
        interpreted_dict = ast.literal_eval(response_dict) # check if readable

        if check_mandatory:
            for key in mandatory_dict_keys:
                if key not in interpreted_dict:
                    return None
    except:
        return None

    response_dict = correct_annotations(response_dict)
    response = f'''Dies ist die gewünschte Annotation in Form eines python Dictionary: 
    {response_dict}'''

    
    return response
    

def create_instruction(text,model='standard'):

    if 'gpt-4' not in model:
        examp = ''' das performativ explizit durch "zum Beispiel", "z.B." oder "beispielsweise" etc. explizit gemacht wird'''
        de_simple_prompt = f'''Im folgenden Textausschnitt wird ein Beispiel gegeben {examp}. Was ist das Beispiel und wofür ist es ein Beispiel?  Was ist die Wiedergabeform in welcher das Beispiel wiedergeben wird, also steht es in indirekter Rede oder wird es vom Erzähler des Textes verwendet (Wenn es in indirekter Rede steht, schreibe die Quelle/den Sprecher in Klammern dahinter)? Kommt das Beispiel in unausgesprochenen Gedanken eines Protagonisten vor? Was ist die Erzählhaltung (personal oder auktorial) des Textes und aus welcher Perspektive wird erzählt (schreibe die Perspektive in Klammern)?
        Kommt es in einem Gedankengang eines Protagonisten vor? In welcher Zeit und in welchem Modus steht das Beispiel und in welchem der Text?
        Antworte in Form eines Python Dictionaries der Form:
        'Beispiel': das Beispiel,
        'Beispiel_für: für was es ein Beispiel ist,
        'Erzählposition': auktorial oder personaler Erzähler,
        'Wiedergabeform': Erzählstimme oder Figurenrede? Wenn Figurenrede: Wer spricht? ist es direkte oder indirekte Rede?,
        'in_gedankengang': ob das Beispiel Teil eines Gedankengangs ist,
        'Modus_Zeit_Beispiel: Modus und Tempus des Beispiels,
        'Modus_Zeit_Text': Modus und Tempus des Texts 

        Dies ist der zu annotierende Textausschnitt: 

        "{text}"

        '''
    else:

        examples = '''Hier einige Beispiele, wie die Dictionaries aussehen sollen:
Beispieleingabe 1): "ürde sie fallen gelassen wie eine heiße Kartoffel. Und wenn ihre Eltern nicht wären, hätte sie auch am Samstag nach Brimleigh gehen können. Steve und die anderen waren an beiden Abenden da gewesen. Doch wenn Yvonne das gemacht hätte, das wusste sie genau, hätte sie am Sonntag Hausarrest bekommen.  Sie saßen im Wohnzimmer auf dem Boden, den Rücken ans Sofa gelehnt. Yvonne war mit Steve allein, die anderen waren unterwegs. Einige von denen, die hier ein und aus gingen, waren seltsame Vögel. Einer zum Beispiel, Magic Jack, sah mit seinem Bart und seinen wilden Augen unheimlich aus, obwohl sie ihn immer nur freundlich erlebt hatte, aber der Schrecklichste von allen war McGarrity, der verrückte Dichter. Zum Glück tauchte er nicht besonders oft auf.  Irgendetwas an McGarrity machte Yvonne nervös. Er war älter als die anderen und hatte ein schmales Gesicht mit pergamentartiger, knittriger Haut und schwarzen Augen. Er trug immer einen schwarzen Hut und einen passenden Umhang und besaß ein Springmesser mit" 
Beispielausgabe 1):{"Beispiel":"Magic Jack","Beispiel_für":"Charakter aus der Gruppe der Figuren, die regelmäßig Yvonne und Steves Umgebung frequentieren und seltsame Vögel waren", "Erzählposition": "personal (Yvonne), dritte Person", "Wiedergabeform":"Erzählstimme", "in_gedankengang":"Nein","Modus_Zeit_Beispiel":"Indikativ Präteritum","Modus_Zeit_Text":"Indikativ Präteritum"}
Beispieleingabe 2):"h den Lagerraum rollten. Das Skelett. Sein erster Instinkt war richtig gewesen. Er musste lernen, seinen Instinkten zu vertrauen. Denn schon als er das Skelett zum ersten Mal gesehen hatte, wusste er, dass etwas Besonderes daran war.     Es war offensichtlich, dass Kommissar Alf Bengtsson von der Polizei in Sollefteä nicht an unbequeme Arbeitszeiten gewöhnt war. Anderseits konnten sein etwas blasses Gesicht und   vor allem   seine extrem unordentliche Frisur mit etwas ganz anderem zu tun haben. Zum Beispiel mit einer Leiche. Sara Svenhagen nahm an, dass Alf Bengtsson noch nie zuvor eine Leiche gesehen hatte. Routine und Gewichtigkeit, die der gute Kommissar sonst ausstrahlte, lösten sich vor dem Körper des dahingeschiedenen Sten Larsson in Wohlgefallen auf. Nicht zuletzt aufgrund der quer über den Hals verlaufenden Wunde. Dieser Hals lag jetzt aufgeklappt im Krankenhaus von Sollefteä, und die Wundränder wurden mit speziellen Klammern auseinandergehalten, ein Anblick, von dem auch Sara Svenhagen ni"
Beispielausgabe 2):{"Beispiel":"Leiche","Beispiel_für":"Mögliche alternative Gründe für die unordentliche Frisur und das blasse Gesicht von Kommissar Alf Bengtsson", "Erzählposition": "auktorial, dritte Person", "Wiedergabeform":"Erzählstimme", "in_gedankengang":"ja","Modus_Zeit_Beispiel":"Indikativ Präteritum","Modus_Zeit_Text":"Indikativ Präteritum"}
Beispieleingabe 3):"lso: In den nächsten Tagen hatte er einige Nachforschungen angestellt. Nichts Bestimmtes natürlich, nichts, was seinen Plan verraten hätte, nur ein paar Sondierungen bei zuverlässigen Leuten, die er kannte. Elementare Frage: Würde eine SpezialFußbremse allgemeine Anwendung finden Ein Dualsystem zum Beispiel, etwas, das man an den Radnaben befestigen könnte »Sie haben sich früh einen Namen gemacht, nicht wahr « sagte Helen. »Wie Shamus.«"
Beispielausgabe 3):{"Beispiel": "Dualsystem", "Beispiel_für": "Eine mögliche Art der SpezialFußbremse, die Cassidy erforscht", "Erzählposition": "personal (er), dritte Person", "Wiedergabeform": "Erzählstimme", "in_gedankengang": "Ja", "Modus_Zeit_Beispiel": "Konjunktiv Präsens", "Modus_Zeit_Text": "Indikativ Präteritum"}
Beispieleingabe 4):"beurteilen. Die so gewonnenen statistisch signifi kanten Daten, erklärte Appelbaum, zeigten eindeutig, dass das Geschlecht ein starkes Bewertungskriterium sei. Es gebe da einige gutdokumentierte, sich selbst verstärkende Denkschleifen zum Beispiel würden sich Leute in Berei chen bewerben, wo bereits »ihresgleichen« tätig sei und sie mehr Aussicht auf Erfolg zu haben meinten. Als Appelbaum zu ihrer Schlussfolgerung kam, hatte Beard den Eindruck, er allein höre ihr noch zu: Das post moderne Publikum "
Beispielausgabe 4):{
"Beispiel": "Leute würden sich in Bereichen bewerben, wo bereits 'ihresgleichen' tätig sei und sie mehr Aussicht auf Erfolg zu haben meinten",
"Beispiel_für": "Gutdokumentierte, sich selbst verstärkende Denkschleifen, die das Geschlecht als ein starkes Bewertungskriterium bestätigen",
"Erzählposition": "personal (Beard), dritte Person",
"Wiedergabeform": "Figurenrede, indirekt (Appelbaum)",
"in_gedankengang": "Nein",
"Modus_Zeit_Beispiel": "Konjunktiv Präteritum",
"Modus_Zeit_Text": "Indikativ Präteritum"
}'''

        de_simple_prompt  = f'''Im folgenden literarischen, narrativen Text wird ein Beispiel verwendet das explizit durch "zum Beispiel", "beispielsweise", "z.B." etc. gekennzeichnet ist. Annotiere dieses Beispiel anhand folgender Fragen: Was ist das Beispiel? Für was steht das Beispiel? Was ist die Erzählposition des Textes (wenn es personal ist, schreibe bitte in Klammern die Perspektive aus der erzählt wird)? 
Wird das Beispiel in der Figurenrede oder von der Erzählstimme verwendet (wenn es Figurenrede ist, schreibe bitte ob es sich um direkte oder indirekte Rede handelt und in Klammern den Sprecher, also zum Beispiel: "Wiedergabeform": "Figurenrede, indirekt (Protagonist); 
wenn es nicht Figurenrede ist, aber die Erzählstimme in indirekter Rede eine Quelle (wie einen inneren Monolog, ein Gerücht oder eine Zeitung etc.) zitiert, schreibe zum Beispiel: "Wiedergabeform": "Erzählstimme, indirekt (Quelle: Zeitung))?
Ist das Beispiel Teil eines Gedankenganges eines Protagonisten? In welcher Zeitform und welchem Modus wird das Beispiel erzählt? In welchem Modus und welcher Zeitform wird der Text insgesamt erzählt? Antworte bitte in Form eines Python-Dictonaries. 
{examples}
Gehe Schritt für Schritt vor: Erstelle eine Liste aller Verben des Texts und schreibe dahinter ihren Tempus und Modus. Was folgt daraus für die Wiedergabeform? Frage anschließend: Was spricht für und gegen einen personalen Erzähler und was für oder gegen einen auktorialen Erzähler? Argumentiere anschließend bezüglich der verbleibenden Fragen. 
Was spricht dafür und dagegen dass es in einem Gedankengang vorkommt (natürlich ist alles was der Erzähler erzählt davor in seinen Gedanken, aber so einfach ist es nicht)? Besonders wichtig ist mir der Punkt  ob es sich um direkt oder indirekt wiedergegebene Figurenrede oder die Erzählinstanz (also sozusagen der Autor) das Beispiel gibt. 
Wenn du dies richtig hast werde ich ein Trinkgeld bezahlen. Schreibe deshalb als letztes in deiner Argumentation noch einmal den Teil des Textes der das Beispiel ist und anschließend "hierbei handelt es sich um direkte/indirekte Rede/ wiedergabe durch die Erzählposition handelt. Im Fall der Beispieleingabe 4 wäre diese Argumentation: 
Das Beispiel kommt im Teil "Es gebe da einige gutdokumentierte, sich selbst verstärkende Denkschleifen zum Beispiel würden sich Leute in Berei chen bewerben, wo bereits »ihresgleichen« tätig sei und sie mehr Aussicht auf Erfolg zu haben meinten." vor und somit handelt es sich aufgrund der Konjunktivkonstruktion um indirekte Figurenrede. 
Außerdem bezieht sich "erklärte Appelbaum" ebenfalls darauf was die Schlussfolgerung unterstützt. Im Fall der Beispieleingabe 2 wäre die Argumentation bezüglich der Wiedergabeform: Das Beispiel steht im Teil "Anderseits konnten sein etwas blasses Gesicht und   vor allem   seine extrem unordentliche Frisur mit etwas ganz anderem zu tun haben. 
Zum Beispiel mit einer Leiche." Es gibt weder ein einleitendes Verb wie "sagen" noch Konjunktiv der auf indirekte Rede hindeuten würde. Es ist auch keine direkte Rede da besagtes Verb des Ausdrucks (Verbum dicendi) fehlt und auch keine Anführungszeichen vorhanden sind. Es wird also von der Erzählinstanz wiedergegeben. Bezüglich des Gedankengangs 
legt der nachfolgende Text jedoch nahe dass sich das Beispiel im Gedankengang von Sara Svenhagen befindet. Denn nun folgt "Sara Svenhagen nahm an, dass Alf Bengtsson noch nie [...] eine Leiche gesehen hatte" womit naheliegt dass sie dies aufgrund des durch das Beispiel erörterte denkt, also dieses Beispiel in den Gedanken hatte.  
        
Verfasse anschließend das Dictionary. 
Zusätzliche Regeln:
1) Wenn es sich um indirekte oder direkte Rede handelt ist es Figurenrede
2) Wenn der Autor das Beispiel selbst verwendet, also selbst ein Beispiel gibt, handelt es sich um die Erzählstimme
3) Wenn das Beispiel in direkter oder indirekter Rede steht ist es nicht in einem Gedankengang 
4) Ein rein innerer Monolog oder die Erzählung über etwas inneres (das aber von niemandem geäußert wurde) ist dagegen ein Gedankengang
5) direkte Rede (direkt) bezeichnet eine direkt wiedergebene Rede. Sie ist meist gekennzeichnet durch Anführungszeichen und ein Verb des sprechens wie sagen etc. Zum Beispiel: Peter sagt "". Es geht aber auch und Julia: "". Bei "Peter sagt er würde zu spät kommen" handelt es sich dagegen umd indirekte Rede (indirekt)
Dies ist der zu annotierende Textausschnitt: 
        
        "{text}"
        '''



        

    return de_simple_prompt



In [73]:
def load_and_concatenate_dfs(filepath):
    if isinstance(filepath, list):
        # Load and concatenate dataframes from all file paths in the list
        dfs = [pd.read_pickle(path) for path in filepath]
        concatenated_df = pd.concat(dfs, ignore_index=True)
    else:
        # Load a single dataframe
        concatenated_df = pd.read_pickle(filepath)

    # Remove rows where the "result" column is None or an empty string
    #concatenated_df = concatenated_df[concatenated_df['result'].notna() & (concatenated_df['result'] != '')]

    return concatenated_df

In [75]:
#filepath = './data/synthetic/annotated_448_bsp_to_synthetic.pkl'

filepath = ['./data/ebooks_conj_df.pkl','./data/ebooks_other_df.pkl','./data/ebooks_presence_df.pkl']

#filepath = './data/synthetic/annotated_2_1209_bsp_to_synthetic_2.pkl'


#anno_df = pd.read_pickle(filepath)

anno_df = load_and_concatenate_dfs(filepath)[['examp_excerpts', 'result']]
# Rename column
anno_df.rename(columns={'result': 'synthetic'}, inplace=True)
anno_df.head()

,examp_excerpts,synthetic
0,"achtet, war aber viel zu high gewesen, um zu b...",In dem vorgelegten Textauszug finden wir ein a...
1,"ekommen sei, habe er so wenige Empfindungen ge...","In dem vorliegenden Textauszug, der komplett i..."
2,"t. Er hatte ihn gebeten, eine vertrauenswür di...",In dem gegebenen Textauszug wird 'zum Beispiel...
3,"nken und schlecht gelaunt, als er endlich kam....",In der gegebenen Passage benutzt der Erzähler ...
4,"hen sollte, wo man nicht so sehr von den Gezei...",In der vorliegenden Passage wird die Redewendu...


In [76]:
anno_df['response'] = anno_df['synthetic'].apply(lambda x: create_response(x,check_mandatory=True))

# Create new 'instruction' column by applying create_instruction function
anno_df['instruction'] = anno_df['examp_excerpts'].apply(lambda x: create_instruction(x,model='standard'))

print(len(anno_df))
# Remove rows where 'result' is None
anno_df.dropna(subset=['response'], inplace=True)

# Reset the index to make it continuous and drop the old index
anno_df.reset_index(drop=True, inplace=True)
print(len(anno_df))

45673
180


In [77]:
anno_df.head()

,examp_excerpts,synthetic,response,instruction
0,ss es ihre Kinder einmal weniger gut haben wür...,"{""Beispiel"": ""das Informationszeitalter"", ""Bei...",Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...
1,o Holger würde die Dynamik der Marktkräfte...,"{\n ""Beispiel"": ""sonnenklar, dass das chinesi...",Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...
2,vor ihnen auf der Erde oder sie saßen irgendw...,"{""Beispiel"": ""Sellerie"", \n ""Beispiel_für"": ""D...",Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...
3,obiltelefonen; sie sahen sich die Fotoalben an...,"{\n ""Beispiel"": ""Was?"",\n ""Beispiel_für"": ""D...",Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...
4,dritten Mal irritierend wurde. Erst besuchte...,"{\n ""Beispiel"": ""Bei feuchtem Wetter"",\n ""Be...",Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...


In [78]:
anno_df.to_pickle('./data/synthetic/gpt3_human_annotated_examples.pkl')

### GPT3.5


In [86]:
anno_df = load_and_concatenate_dfs(['./data/synthetic/gpt4_annotated_examples_2_cleaned.pkl','./data/synthetic/gpt4_annotated_examples_1_cleaned.pkl'])

In [87]:
# Shuffle the DataFrame
anno_df = anno_df.sample(frac=1).reset_index(drop=True)

# Split the DataFrame into training and evaluation sets
train_df = anno_df[:int(0.92 * len(anno_df))]
eval_df = anno_df[int(0.92 * len(anno_df)):]

def dataframe_to_json(df):
    json_data = []
    for _, row in df.iterrows():
        message_block = [
            {"role": "system", "content": "Du bist der führende deutsche Literaturwissenschaftler Prof. Dr. Andreas Huysse. Deine Tätigkeit ist in der Quantitativen Literaturwissenschaft. Deine Aufgabe ist es Textstellen zu annotieren."},
            {"role": "user", "content": row['instruction']},
            {"role": "assistant", "content": row['response']}
        ]
        json_data.append({"messages": message_block})
    return json_data#json.dumps(json_data, indent=2)

# Converting DataFrames to JSON
json_train_output = dataframe_to_json(train_df)
json_eval_output = dataframe_to_json(eval_df)

# File paths
train_output_file_path = "./data/synthetic/finetune_gpt3.5_clean_train.jsonl"
eval_output_file_path = "./data/synthetic/finetune_gpt3.5_clean_eval.jsonl"

# Writing to files
with open(train_output_file_path, 'w',encoding='utf-8') as file:
    for item in json_train_output:
        json.dump(item, file)
        file.write('\n')

with open(eval_output_file_path, 'w',encoding='utf-8') as file:
    for item in json_eval_output:
        json.dump(item, file)
        file.write('\n')

In [82]:
anno_df.head()

,level_0,index,Id,Md5Hash,Title,Authors,Series,Edition,Language,Year,...,narrated_examps,tense_modus,first_person,praesens,konjunktiv,prompt,synthetic,response,instruction,result
0,231434,1415824,1415825,301129A542A449C760796D3616D88103,Meine russische Großmutter und ihr amerikanisc...,"Shalev, Meir",,,German,2014,...,"oßmutter in Acht nehmen, nicht Großmutter vor ...","([Präteritum, Präsens, Präteritum, Präteritum,...",False,False,False,,Liste der Verben im Textausschnitt und deren T...,Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...,NaN
1,22554,123219,123220,8523F74F59AED4E8E9E945C00EA7A301,Werke Band 2,"Roth, Joseph",,,German,0,...,"hotechnischen Prüfung, die der Vortragende, ei...","([Präsens, Präteritum, Präteritum, Präteritum,...",True,False,False,,"Um die Aufgabe zu erfüllen, analysiere ich zun...",Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...,NaN
2,22250,122915,122916,1D49F641D2F6BB6B9B2AB7E07AAC5E53,Unbekannte Realität Band2,"Roberts, Jane",,,German,0,...,e andere -------------- zubewegt. Dies e repr...,"([Präsens, Präsens, Präsens, Präsens, Präsens,...",False,True,False,,"Um die gewünschte Annotation zu erstellen, ana...",Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...,NaN
3,13299,113942,113943,9AB058078826936E575ADB0E9B2CC011,Was vom Tage übrigblieb,"Ishiguro, Kazuo",,,German,0,...,"n und Gelegenheit gehabt hatte, meine Gedanke...","([Präteritum, Präsens, Präsens, Präteritum, Pr...",True,False,False,,Um die gewünschten Informationen zu extrahiere...,Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...,NaN
4,151777,1223524,1223525,12EA3B75A06E320B7E0AE3D5728D95DD,Ideen. Eine Kulturgeschichte von der Entdeckun...,"Watson, Peter",,,German,2013,...,----------------------------------------------...,"([Präteritum, Präteritum, Präteritum, Präterit...",False,True,False,,Um die gewünschten Informationen zu extrahiere...,Dies ist die gewünschte Annotation in Form ein...,Im folgenden Textausschnitt wird ein Beispiel ...,NaN


In [81]:
len(anno_df)

1481

In [83]:
print(anno_df.loc[1200,'instruction'] )

Im folgenden Textausschnitt wird ein Beispiel gegeben  das performativ explizit durch "zum Beispiel", "z.B." oder "beispielsweise" etc. explizit gemacht wird. Was ist das Beispiel und wofür ist es ein Beispiel?  Was ist die Wiedergabeform in welcher das Beispiel wiedergeben wird, also steht es in indirekter Rede oder wird es vom Erzähler des Textes verwendet (Wenn es in indirekter Rede steht, schreibe die Quelle/den Sprecher in Klammern dahinter)? Kommt das Beispiel in unausgesprochenen Gedanken eines Protagonisten vor? Was ist die Erzählhaltung (personal oder auktorial) des Textes und aus welcher Perspektive wird erzählt (schreibe die Perspektive in Klammern)?
        Kommt es in einem Gedankengang eines Protagonisten vor? In welcher Zeit und in welchem Modus steht das Beispiel und in welchem der Text?
        Antworte in Form eines Python Dictionaries der Form:
        'Beispiel': das Beispiel,
        'Beispiel_für: für was es ein Beispiel ist,
        'Erzählposition': auktorial od

In [55]:
print(anno_df.loc[1600,'response'] )


Dies ist die gewünschte Annotation in Form eines python Dictionary: 
    {
  "Beispiel": "viele frische Kräuter (zum Beispiel Petersilie, Kerbel, Schnittlauch)",
  "Beispiel_für": "Zutaten, die dem Gericht Geschmack verleihen, wenn keine Gemüsebrühe zur Hand ist",
  "Erzählposition": "auktorial, dritte Person",
  "Wiedergabeform": "Erzählstimme",
  "in_gedankengang": "Nein",
  "Modus_Zeit_Beispiel": "Indikativ Präsens",
  "Modus_Zeit_Text": "Indikativ Präsens"
}
